# AV388

Updating the dataset: P117540100000

* S/N of LiF 1 is low because of detector shutdowns. 
* Order of preference is LiF 2, SiC 2, LiF 1, SiC 1. 
* No rescaling needed.

COMMENT This NVO file has been reconstructed from the corresponding ALL file.<br>
COMMENT Order of selection is LiF 2, SiC 2, LiF 1, SiC 1.


In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E13

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Read header keywords.

filename = 'p117540100000all4ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# The guide channel was LiF 1.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed combined spectra.  
# Note that spectra are scaled to have values near unity.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC2B') 
ax.legend()
ax.set_ylim([0,10])

# The fluxes are OK.  Let's look at S/N. 

In [ ]:
# Plot the (unsmoothed) error arrays.

f, ax = plt.subplots()  
ax.step(sic1a.spectral_axis, sic1a_data['error'], label='SiC 1A')
ax.step(lif1a.spectral_axis, lif1a_data['error'], label='LiF 1A') 
ax.step(sic2b.spectral_axis, sic2b_data['error'], label='SiC 2B')
ax.step(lif2b.spectral_axis, lif2b_data['error'], label='LiF 2B') 
ax.legend()

# So order of preference is LiF 2, SiC 2, LiF 1, SiC 1.

In [ ]:
# Let's see what's happening at long wavelengths.

f, ax = plt.subplots()  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF1B') 
ax.legend()
ax.set_ylim([0,8])

# LiF 1B has a worm, but it's not in a region that we'll be using.

In [ ]:
# Compare the long-wavelength segments with a COS spectrum of the same star.

cos = Spectrum1D.read("lb2a01010_x1dsum.fits", format="HST/COS")
cos_bsmooth = box_smooth(cos, width=15)

f, ax = plt.subplots()
ax.plot(cos_bsmooth.spectral_axis, norm * cos_bsmooth.flux, label='COS', color='g')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
# ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,8])

# Looks good.  No need to rescale the spectrum.

In [ ]:
# At short wavelengths...

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([900, 1010])
ax.set_ylim([0,10])

# The fluxes are OK.

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 2B.

shift_lif1 = compute_shift(lif1a_data, lif2b_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif2b_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif2b_data, 1040, 1070)

print ('Shift LiF 1 by ', shift_lif1, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif1 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif2a_shift = lif2a
lif2b_shift = lif2b
lif1a_shift = Spectrum1D(spectral_axis=lif1a.spectral_axis + shift_lif1 * u.AA, flux=lif1a.flux)
lif1b_shift = Spectrum1D(spectral_axis=lif1b.spectral_axis + shift_lif1 * u.AA, flux=lif1b.flux)
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([0,10])

In [ ]:
# Again, order of preference is LiF 2, SiC 2, LiF 1, SiC 1.

filename = 'p117540100000nvo4ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 917.5))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 917.5))
data['flux'][g] = sic1b_data['flux'][h]
data['error'][g] = sic1b_data['error'][h]

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 917.5) & (data['wave'] < 990))
h = np.where((sic2a_wave > 917.5) & (sic2a_wave < 990))
data['flux'][g] = sic2a_data['flux'][h]
data['error'][g] = sic2a_data['error'][h]

# Scale LiF 2B
lif2b_wave = lif2b_shift.spectral_axis.value
g = np.where((data['wave'] > 990) & (data['wave'] < 1074))
h = np.where((lif2b_wave > 990) & (lif2b_wave < 1074))
data['flux'][g] = lif2b_data['flux'][h]
data['error'][g] = lif2b_data['error'][h]

# Scale SiC 2B 
sic2b_wave = sic2b_shift.spectral_axis.value
g = np.where((data['wave'] > 1074) & (data['wave'] < 1087))
h = np.where((sic2b_wave > 1074) & (sic2b_wave < 1087))
data['flux'][g] = sic2b_data['flux'][h]
data['error'][g] = sic2b_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087) & (data['wave'] < 1180))
h = np.where((lif2a_wave > 1087) & (lif2a_wave < 1180))
data['flux'][g] = lif2a_data['flux'][h]
data['error'][g] = lif2a_data['error'][h]

# Scale LiF 1B
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1180) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1180) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = lif1b_data['flux'][h]
data['error'][g] = lif1b_data['error'][h]

hdr['comment'] = ''
hdr['comment'] = 'File updated 21 April 2022'
hdr['comment'] = 'This NVO file has been reconstructed from the corresponding ALL file.'
hdr['comment'] = 'Order of preference is LiF 2, SiC 2, LiF 1, SiC 1.'

f.writeto('level0_p117540100000nvo4ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'p117540100000nvo4ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_p117540100000nvo4ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.step(new['wave'], new['flux'], label='NEW FLUX')

ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')

ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,1.2E-12])